In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torch.nn.functional as F
import sklearn
import torchvision
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import PIL
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2
import seaborn as sns
import glob
from pathlib import Path
torch.manual_seed(1)
np.random.seed(1)

In [2]:
train_labels = pd.read_csv('../input/retinal-disease-classification/Training_Set/Training_Set/RFMiD_Training_Labels.csv')
valid_labels = pd.read_csv('../input/retinal-disease-classification/Evaluation_Set/Evaluation_Set/RFMiD_Validation_Labels.csv') 
test_labels = pd.read_csv('../input/retinal-disease-classification/Test_Set/Test_Set/RFMiD_Testing_Labels.csv')

In [3]:
train_files = []
valid_files = []
test_files = []
for file in os.listdir('../input/retinal-disease-classification/Training_Set/Training_Set/Training'):
    train_files.append(file)
    
for file in os.listdir('../input/retinal-disease-classification/Evaluation_Set/Evaluation_Set/Validation'):
    valid_files.append(file)
    
for file in os.listdir('../input/retinal-disease-classification/Test_Set/Test_Set/Test'):
    test_files.append(file)

In [4]:
train_ids = []
for element in train_files:
    train_ids.append(element.split('.')[0])
    
valid_ids = []
for element in valid_files:
    valid_ids.append(element.split('.')[0])

test_ids = []
for element in test_files:
    test_ids.append(element.split('.')[0])

In [5]:
train_ids = pd.Series(train_ids, name='ids')
train_files = pd.Series(train_files, name='filenames')
train_files = pd.concat([train_ids, train_files], axis=1)

valid_ids = pd.Series(valid_ids, name='ids')
valid_files = pd.Series(valid_files, name='filenames')
valid_files = pd.concat([valid_ids, valid_files], axis=1)

test_ids = pd.Series(test_ids, name='ids')
test_files = pd.Series(test_files, name='filenames')
test_files = pd.concat([test_ids, test_files], axis=1)

In [6]:
train_files['ids'] = train_files['ids'].astype('int64')
valid_files['ids'] = valid_files['ids'].astype('int64')
test_files['ids'] = test_files['ids'].astype('int64')

In [7]:
train_df = pd.merge(train_labels, train_files, left_on='ID', right_on='ids')
valid_df = pd.merge(valid_labels, valid_files, left_on='ID', right_on='ids')
test_df = pd.merge(test_labels, test_files, left_on='ID', right_on='ids')
train_df

,ID,Disease_Risk,DR,ARMD,MH,DN,MYA,BRVO,TSLN,ERM,...,CF,VH,MCA,VS,BRAO,PLQ,HPED,CL,ids,filenames
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1.png
1,2,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2.png
2,3,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,3.png
3,4,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,4,4.png
4,5,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5,5.png
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1915,1916,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1916,1916.png
1916,1917,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1917,1917.png
1917,1918,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1918,1918.png
1918,1919,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1919,1919.png


In [8]:
train_df.drop(columns='ids')
valid_df.drop(columns='ids')
test_df.drop(columns='ids')

,ID,Disease_Risk,DR,ARMD,MH,DN,MYA,BRVO,TSLN,ERM,...,PTCR,CF,VH,MCA,VS,BRAO,PLQ,HPED,CL,filenames
0,1,1,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1.png
1,2,1,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,2.png
2,3,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3.png
3,4,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4.png
4,5,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5.png
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635,636,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,636.png
636,637,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,637.png
637,638,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,638.png
638,639,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,639.png


In [9]:
train_df['full_file_paths'] = '../input/retinal-disease-classification/Training_Set/Training_Set/Training/' + train_df['filenames']
valid_df['full_file_paths'] = '../input/retinal-disease-classification/Evaluation_Set/Evaluation_Set/Validation/' + valid_df['filenames']
test_df['full_file_paths'] = '../input/retinal-disease-classification/Test_Set/Test_Set/Test/' + test_df['filenames']

In [10]:
class RetinalDisease(torch.utils.data.Dataset):
    def __init__(self, df=train_df, transform=transforms.Compose([transforms.ToTensor()])):
        self.df = df
        self.transform = transform
        
    def __len__(self):
        length = len(self.df)
        return length
        
    def __getitem__(self, idx):
        img_path = self.df.iloc[idx, -1]
        label = torch.tensor(train_df.iloc[idx, 1:-3])
        image = Image.open(img_path).convert('RGB')
        img = np.array(image)
        image = self.transform(image=img)["image"]
        return image, label

In [11]:
train_transforms = A.Compose([
    A.Resize(1424, 2144),
    A.HorizontalFlip(),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

test_transforms = A.Compose([
    A.Resize(1424, 2144),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

In [12]:
train_dataset = RetinalDisease(transform=train_transforms)
valid_dataset = RetinalDisease(df=valid_df, transform=test_transforms)
test_dataset = RetinalDisease(df=test_df, transform=test_transforms)

In [13]:
batch_size = 2

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=10)

In [14]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [15]:
model = torchvision.models.vgg16(pretrained=True)
model.classifier[6] = nn.Linear(4096,46)
print(model)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [16]:
optimizer = torch.optim.Adam(model.parameters(), weight_decay=0.0005, lr=0.0001)

criterion = nn.BCEWithLogitsLoss(reduction='sum')

lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=8, cooldown=10, verbose=True)

model = model.to(device)
criterion = criterion.to(device)

In [17]:
epochs = 5

total_train_loss = []
total_valid_loss = []
best_valid_loss = np.Inf

for epoch in range(epochs): 
    print('Epoch: ', epoch + 1)
    temp=1
    train_loss = []
    valid_loss = []
    train_correct = 0
    train_total = 0
    valid_correct = 0
    valid_total = 0
    for image, target in train_loader:
        model.train()
        image, target = image.to(device), target.to(device)
        output = model(image)
        target = target.float()
         
        loss = criterion(output, target)
        train_loss.append(loss.item())
        print("Epoch:",epoch+1," iteration:", temp," Loss:",loss.item())
        loss.backward()
        optimizer.step()
        temp+=1
    for image, target in valid_loader:
        with torch.no_grad():
            model.eval()
            optimizer.zero_grad()
            image, target = image.to(device), target.to(device)
            output = model(image)
            target = target.float()
            loss = criterion(output, target)
            valid_loss.append(loss.item())
            
    epoch_train_loss = np.mean(train_loss)
    epoch_valid_loss = np.mean(valid_loss)
    print(f'Epoch {epoch + 1}, train loss: {epoch_train_loss:.4f}, valid loss: {epoch_valid_loss:.4f}')
    if epoch_valid_loss < best_valid_loss:
        torch.save(model.state_dict(), 'retinal_disease.pt')
        print('Model improved. Saving model.')
        best_valid_loss = epoch_valid_loss
        
    lr_scheduler.step(epoch_valid_loss)
    total_train_loss.append(epoch_train_loss)
    total_valid_loss.append(epoch_valid_loss)

Epoch:  1
Epoch: 1  iteration: 1  Loss: 63.660274505615234
Epoch: 1  iteration: 2  Loss: 61.72114944458008
Epoch: 1  iteration: 3  Loss: 54.805747985839844
Epoch: 1  iteration: 4  Loss: 44.86554718017578
Epoch: 1  iteration: 5  Loss: 28.438556671142578
Epoch: 1  iteration: 6  Loss: 28.419002532958984
Epoch: 1  iteration: 7  Loss: 23.501239776611328
Epoch: 1  iteration: 8  Loss: 33.598289489746094
Epoch: 1  iteration: 9  Loss: 26.119550704956055
Epoch: 1  iteration: 10  Loss: 16.305526733398438
Epoch: 1  iteration: 11  Loss: 9.55862045288086
Epoch: 1  iteration: 12  Loss: 10.340761184692383
Epoch: 1  iteration: 13  Loss: 11.500847816467285
Epoch: 1  iteration: 14  Loss: 20.390642166137695
Epoch: 1  iteration: 15  Loss: 13.879264831542969
Epoch: 1  iteration: 16  Loss: 14.833911895751953
Epoch: 1  iteration: 17  Loss: 10.538901329040527
Epoch: 1  iteration: 18  Loss: 13.158790588378906
Epoch: 1  iteration: 19  Loss: 8.466764450073242
Epoch: 1  iteration: 20  Loss: 9.049825668334961
Epoch

In [18]:
torch.cuda.empty_cache()
torch.cuda.synchronize()
